# **Input and Output Ports of the Hardware Station**


In [14]:
import numpy as np
from IPython.display import HTML
from pydrake.all import StartMeshcat, namedview
from pydrake.systems.framework import GenerateHtml

In [15]:
from manipulation.exercises.grader import Grader
from manipulation.exercises.robot.test_hardware_station_io import (
    TestHardwareStationIO,
)
from manipulation.station import MakeHardwareStation, load_scenario

In [16]:
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7005


## Access System Input/Output Values
In this exercise, you will explore the [`HardwareStation`](http://manipulation.mit.edu/robot.html#hardware_station) that was mentioned during the lecture. You should recall that the orange ports are the ones that do not exist for the actual hardware platform.

<html>
<table align="center" cellpadding="0" cellspacing="0"><tbody><tr align="center"><td style="vertical-align:middle"><table cellspacing="0" cellpadding="0"><tbody><tr><td align="right" style="padding:5px 0px 5px 0px">iiwa.position→</td></tr><tr><td align="right" style="padding:5px 0px 5px 0px">iiwa.feedforward_torque (optional)→</td></tr><tr><td align="right" style="padding:5px 0px 5px 0px">wsg.position→</td></tr><tr><td align="right" style="padding:5px 0px 5px 0px">wsg.force_limit (optional)→</td></tr></tbody></table></td><td align="center" style="border:solid;padding-left:20px;padding-right:20px;vertical-align:middle" bgcolor="#F0F0F0"><a href="https://github.com/RussTedrake/manipulation/blob/ceb817b527cbf1826c5b9a573ffbef415cb0f013/manipulation/scenarios.py#L453">HardwareStation</a></td><td style="vertical-align:middle"><table cellspacing="0" cellpadding="0"><tbody><tr><td align="left" style="padding:5px 0px 5px 0px">→ iiwa.position_commanded</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ iiwa.position_measured</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ iiwa.velocity_estimated</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ iiwa.state_estimated</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ iiwa.torque_commanded</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ iiwa.torque_measured</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ iiwa.torque_external</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ wsg.state_measured</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ wsg.force_measured</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ camera_[NAME].rgb_image</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ camera_[NAME].depth_image</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ <b style="color:orange">camera_[NAME].label_image</b></td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ ...</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ camera_[NAME].rgb_image</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ camera_[NAME].depth_image</td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ <b style="color:orange">camera_[NAME].label_image</b></td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ <b style="color:orange">query_object</b></td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ <b style="color:orange">contact_results</b></td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ <b style="color:orange">plant_continuous_state</b></td></tr><tr><td align="left" style="padding:5px 0px 5px 0px">→ <b style="color:orange">body_poses</b></td></tr></tbody></table></td></tr></tbody></table>
</html>

Now we construct a `HardwareStation` object and finalize the system setting. To get a sense of what this hardware station scenario looks like, you can open the meshcat viewer from the generated link as usual. There should be a bookshelf and a Kuka arm with a gripper attached (it might take a few seconds to load).

In [17]:
scenario_data = """
directives:
- add_directives:
    file: package://manipulation/clutter.dmd.yaml
model_drivers:
    iiwa: !IiwaDriver
      hand_model_name: wsg
    wsg: !SchunkWsgDriver {}
"""
scenario = load_scenario(data=scenario_data)
station = MakeHardwareStation(scenario, meshcat=meshcat)
context = station.CreateDefaultContext()

station.GetInputPort("wsg.position").FixValue(context, [0.1])
station.ForcedPublish(context)

[**Context**](https://drake.mit.edu/pydrake/pydrake.systems.framework.html?highlight=context#pydrake.systems.framework.Context_) is an abstract class template that represents all the typed values that are used in a System’s computations: time, numeric-valued input ports, numerical state, and numerical parameters. There are also type-erased abstract state variables, abstract-valued input ports, abstract parameters, and a double accuracy setting. It is important to note that a **Context** is designed to be used only with the System that created it. State and Parameter data can be copied between contexts for compatible systems as necessary. One of the most common mistakes is to pass the wrong context. Most methods in Drake should throw an error if you pass a context from the wrong system, but a few do not perform this check yet.

In the cell below, we first create a root context from the diagram, and then we retrieve the contexts of the subsystems from the root context.

In [18]:
# initialize context
station_context = station.CreateDefaultContext()
plant = station.GetSubsystemByName("plant")
plant_context = plant.GetMyContextFromRoot(station_context)

In this exercise, you will familiarize yourself with the input and output mechanism from the hardware station system. Remember you can always generate a schematic view of your system by running the cell below. By clicking the "+" sign on the manipulation_station, you can get a more detailed view of the diverse modules within the hardware station. (You might need to run the cell twice to see the diagram)

In [19]:
gojs = '<script src="https://unpkg.com/gojs/release/go.js"></script>'
HTML(gojs + GenerateHtml(station))

/var/folders/c0/jfh1p3f53dg3c5jwvx1gfvx80000gp/T/ipykernel_67424/3340321670.py:2: DrakeDeprecationWarning: (Deprecated.)

Deprecated:
    This feature has been removed from Drake. This will be removed
    from Drake on or after 2024-01-01.
  HTML(gojs + GenerateHtml(station))


Now if we set the joint position of the Kuka arm, we should expect to get the same values from the `iiwa.position_measured port`, which can be found from the output ports of `station` object. Below we demonstrate how this can be done using Drake's syntax. You may also find it useful to review the `System` class documentation [here](https://drake.mit.edu/pydrake/pydrake.systems.framework.html?highlight=output_port#pydrake.systems.framework.System_).

In [20]:
# provide initial states
q0 = np.array([-1.57, 0.1, 0, -1.2, 0, 1.6, 0])
iiwa = plant.GetModelInstanceByName("iiwa")
# set the joint positions of the kuka arm
plant.SetPositions(plant_context, iiwa, q0)
# examine the output port
station.GetOutputPort("iiwa.position_measured").Eval(station_context)

array([-1.57,  0.1 ,  0.  , -1.2 ,  0.  ,  1.6 ,  0.  ])

Note that the [output port](https://drake.mit.edu/pydrake/pydrake.systems.framework.html?highlight=outputport#pydrake.systems.framework.OutputPort) named `iiwa.position_measured` is first retrieved from the station and then evaluated using **Eval** method. This is a very common approach to read the values of a selected output port.


*Drake advice:* use `nameview` to refer to positions by name.

In [21]:
IiwaPositionView = namedview(
    "IiwaPositions",
    plant.GetPositionNames(iiwa, always_add_suffix=False),
)
q0 = IiwaPositionView.Zero()
q0.iiwa_joint_1 = -1.57
q0.iiwa_joint_2 = 0.1
q0.iiwa_joint_4 = -1.2
q0.iiwa_joint_6 = 1.6
print(q0)
plant.SetPositions(plant_context, iiwa, q0[:])

IiwaPositions(iiwa_joint_1=-1.57, iiwa_joint_2=0.1, iiwa_joint_3=0, iiwa_joint_4=-1.2, iiwa_joint_5=0, iiwa_joint_6=1.6, iiwa_joint_7=0)


Sometimes, when working directly with a `MultibodyPlant`, you may retrieve the joint angles directly from the **plant**. But this will not work when we're running on real hardware. Use the input and output ports to properly leverage the "sim 2 real" abstraction.

In [22]:
print(plant.GetPositions(plant_context, iiwa))

[-1.57  0.1   0.   -1.2   0.    1.6   0.  ]


# Exercise a: Code Submission
Now, it's your time to code! Use **GetOutputPort** and **Eval** to retrieve the joint velocities from the "iiwa_velocity_estimated" output port. Note that we have set the velocities for you.

In [23]:
plant.SetVelocities(
    plant_context,
    iiwa,
    np.zeros(
        7,
    ),
)

Below, `get_velocity(station, station_context)` is the function you must modify to query values from "iiwa_velocity_estimated".

In [24]:
def get_velocity(station, station_context):
    """
    fill in your code in this method
    """
    velocity_estimated = station.GetOutputPort("iiwa.velocity_estimated").Eval(station_context)
    return velocity_estimated

Please note that the `iiwa.position_commanded` and the `iiwa.position_measured` are NOT the same variable. The `iiwa.position_commanded` are the commanded positions sent to the robot, whereas the `iiwa.positions` are the current positions of the simulated robot. We also expect to have different values for the feed-forward torque (system input) and the commanded torque (system output). Next, you will investigate why. First, let us provide a zero feed-forward torque to the `iiwa.feedforward_torque` port.

In [25]:
station.GetInputPort("iiwa.feedforward_torque").FixValue(
    station_context,
    np.zeros(
        7,
    ),
)
tau_no_ff = station.GetOutputPort("iiwa.torque_commanded").Eval(
    station_context
)
print(
    "feedforward torque: {}".format(
        np.zeros(
            7,
        )
    )
)
print("commanded torque with no feedforward torque:{}".format(tau_no_ff))

feedforward torque: [0. 0. 0. 0. 0. 0. 0.]
commanded torque with no feedforward torque:[ 215.83027562 -285.82565154  194.20019567  258.26699515   36.90660859
 -152.0079284   -73.75860787]


Now try the same experiment with a non-zero feedforward torque as shown below.

In [26]:
tau_ff = np.linspace(3.1, 3.7, 7)
print("feedforward torque: {}".format(tau_ff))
station.GetInputPort("iiwa.feedforward_torque").FixValue(
    station_context, tau_ff
)
torque_commanded = station.GetOutputPort("iiwa.torque_commanded").Eval(
    station_context
)
print("the commanded torque: {}".format(torque_commanded))

feedforward torque: [3.1 3.2 3.3 3.4 3.5 3.6 3.7]
the commanded torque: [ 218.93027562 -282.62565154  197.50019567  261.66699515   40.40660859
 -148.4079284   -70.05860787]
